In [6]:
import os
import requests
import zipfile
import shutil
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# --- 1. CLEAN UP PREVIOUS ATTEMPTS ---
if os.path.exists('cats_and_dogs_filtered.zip'):
    os.remove('cats_and_dogs_filtered.zip')
if os.path.exists('cats_and_dogs_filtered'):
    shutil.rmtree('cats_and_dogs_filtered')

# --- 2. DOWNLOAD WITH BROWSER HEADERS ---
url = "https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip"
dest = "cats_and_dogs_filtered.zip"

print(f"Downloading {url}...")

# This 'headers' part is the magic trick to bypass 403 errors
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'}

response = requests.get(url, headers=headers, stream=True)

if response.status_code == 200:
    with open(dest, 'wb') as f:
        for chunk in response.iter_content(chunk_size=8192):
            f.write(chunk)
    print("Download successful!")

    # --- 3. EXTRACT ---
    print("Extracting...")
    with zipfile.ZipFile(dest, 'r') as zip_ref:
        zip_ref.extractall('.')
    print("Dataset ready.")
else:
    print(f"Failed to download. Status code: {response.status_code}")

Using device: cuda
Failed to download. Status code: 403


In [7]:
# --- CIFAR-10 Loading ---
transform_cifar = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

print("Loading CIFAR-10...")
train_cifar = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform_cifar)
test_cifar = datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_cifar)

train_loader_cifar = DataLoader(train_cifar, batch_size=64, shuffle=True)
test_loader_cifar = DataLoader(test_cifar, batch_size=64, shuffle=False)

# --- Cats vs Dogs Loading ---
transform_dogs = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# The extracted folder is 'cats_and_dogs_filtered/train'
train_dir = './cats_and_dogs_filtered/train'

if os.path.exists(train_dir):
    print("Loading Cats vs Dogs...")
    full_dataset = datasets.ImageFolder(root=train_dir, transform=transform_dogs)

    # Split 80% Train, 20% Test
    train_size = int(0.8 * len(full_dataset))
    test_size = len(full_dataset) - train_size
    train_dogs, test_dogs = torch.utils.data.random_split(full_dataset, [train_size, test_size])

    train_loader_dogs = DataLoader(train_dogs, batch_size=32, shuffle=True)
    test_loader_dogs = DataLoader(test_dogs, batch_size=32, shuffle=False)
    print("Cats vs Dogs loaded successfully.")
else:
    print("Error: Dataset folder not found. Check Step 1.")

Loading CIFAR-10...
Error: Dataset folder not found. Check Step 1.


In [8]:
import torch.nn as nn
import torch.optim as optim
import torchvision
import itertools
import copy

class CustomCNN(nn.Module):
    def __init__(self, num_classes, activation="relu"):
        super(CustomCNN, self).__init__()

        if activation == "relu":
            self.act = nn.ReLU()
        elif activation == "tanh":
            self.act = nn.Tanh()
        elif activation == "leaky":
            self.act = nn.LeakyReLU(0.01)

        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            self.act,
            nn.MaxPool2d(2, 2),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            self.act,
            nn.MaxPool2d(2, 2),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            self.act,
            nn.MaxPool2d(2, 2)
        )

        if num_classes == 10:
            self.flatten_dim = 128 * 8 * 8
        else:
            self.flatten_dim = 128 * 16 * 16

        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(self.flatten_dim, 256),
            self.act,
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

def init_weights(model, init_type):
    for m in model.modules():
        if isinstance(m, (nn.Conv2d, nn.Linear)):
            if init_type == "xavier":
                nn.init.xavier_uniform_(m.weight)
            elif init_type == "kaiming":
                nn.init.kaiming_uniform_(m.weight, nonlinearity='relu')
            elif init_type == "random":
                nn.init.normal_(m.weight, mean=0.0, std=0.02)

def train_epoch(model, loader, criterion, optimizer):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    return running_loss / len(loader), correct / total

def eval_model(model, loader, criterion):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return correct / total

In [9]:
def run_full_experiment(dataset_name, train_loader, test_loader, num_classes):
    print(f"\n STARTING EXPERIMENT: {dataset_name}\n")

    activations = ["relu", "tanh", "leaky"]
    inits = ["xavier", "kaiming", "random"]
    optimizers_list = ["sgd", "adam", "rmsprop"]

    best_acc = 0.0
    best_config = ""
    best_model_wts = None

    criterion = nn.CrossEntropyLoss()


    for act, init, opt in itertools.product(activations, inits, optimizers_list):
        print(f"Testing Config: Act={act} | Init={init} | Opt={opt}")

        model = CustomCNN(num_classes=num_classes, activation=act).to(device)
        init_weights(model, init)

        if opt == "sgd":
            optimizer = optim.SGD(model.parameters(), lr=0.01)
        elif opt == "adam":
            optimizer = optim.Adam(model.parameters(), lr=0.001)
        else:
            optimizer = optim.RMSprop(model.parameters(), lr=0.001)

        for epoch in range(1):
            train_loss, train_acc = train_epoch(model, train_loader, criterion, optimizer)
            val_acc = eval_model(model, test_loader, criterion)

        print(f"  -> Final Validation Acc: {val_acc:.4f}")

        if val_acc > best_acc:
            best_acc = val_acc
            best_config = f"{act}_{init}_{opt}"
            best_model_wts = copy.deepcopy(model.state_dict())

    print(f"\n*** BEST CUSTOM MODEL: {best_config} (Acc: {best_acc:.4f}) ***")

    torch.save(best_model_wts, f'best_cnn_{dataset_name.replace(" ", "")}.pth')

    print(f"\n--- Running ResNet-18 Transfer Learning on {dataset_name} ---")
    resnet = torchvision.models.resnet18(pretrained=True)


    for param in resnet.parameters():
        param.requires_grad = False

    num_ftrs = resnet.fc.in_features
    resnet.fc = nn.Linear(num_ftrs, num_classes)
    resnet = resnet.to(device)

    res_optimizer = optim.Adam(resnet.fc.parameters(), lr=0.001)

    for epoch in range(1):
        _, _ = train_epoch(resnet, train_loader, criterion, res_optimizer)
        res_acc = eval_model(resnet, test_loader, criterion)
        print(f"  ResNet Epoch {epoch+1} Acc: {res_acc:.4f}")

    print(f"\nFINAL COMPARISON ({dataset_name}):")
    print(f"Custom CNN Best: {best_acc:.4f}")
    print(f"ResNet-18:       {res_acc:.4f}")

run_full_experiment("CIFAR-10", train_loader_cifar, test_loader_cifar, 10)

if 'train_loader_dogs' in globals():
    run_full_experiment("Cats vs Dogs", train_loader_dogs, test_loader_dogs, 2)
else:
    print("Skipping Cats vs Dogs (Loader not found).")


 STARTING EXPERIMENT: CIFAR-10

Testing Config: Act=relu | Init=xavier | Opt=sgd
  -> Final Validation Acc: 0.5031
Testing Config: Act=relu | Init=xavier | Opt=adam
  -> Final Validation Acc: 0.4429
Testing Config: Act=relu | Init=xavier | Opt=rmsprop
  -> Final Validation Acc: 0.3639
Testing Config: Act=relu | Init=kaiming | Opt=sgd
  -> Final Validation Acc: 0.4976
Testing Config: Act=relu | Init=kaiming | Opt=adam
  -> Final Validation Acc: 0.3896
Testing Config: Act=relu | Init=kaiming | Opt=rmsprop
  -> Final Validation Acc: 0.3827
Testing Config: Act=relu | Init=random | Opt=sgd
  -> Final Validation Acc: 0.5131
Testing Config: Act=relu | Init=random | Opt=adam
  -> Final Validation Acc: 0.5392
Testing Config: Act=relu | Init=random | Opt=rmsprop
  -> Final Validation Acc: 0.4799
Testing Config: Act=tanh | Init=xavier | Opt=sgd
  -> Final Validation Acc: 0.5009
Testing Config: Act=tanh | Init=xavier | Opt=adam
  -> Final Validation Acc: 0.4978
Testing Config: Act=tanh | Init=xav

/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
100%|██████████| 44.7M/44.7M [00:00<00:00, 186MB/s]


  ResNet Epoch 1 Acc: 0.6276

FINAL COMPARISON (CIFAR-10):
Custom CNN Best: 0.5650
ResNet-18:       0.6276
Skipping Cats vs Dogs (Loader not found).
